In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# Set random seed for reproducibility
rng = np.random.default_rng(42)

# Utility für Aufgaben
def todo():
    raise NotImplementedError("In dieser Zelle gibt es noch mindestens ein TODO!")

# Ableiten als inverses Problem

In diesem Notebook sehen wir, warum das Ableiten einer normalen, eindimensionalen Funktion ein inverses Problem ist.
Zu Beginn definieren wir uns `true_function`. Das Auswerten dieser Funktion auf einem Gitter produziert `x`, die korrekte Lösung des inversen Problems ('ground truth').

Danach integrieren wir `x` und produzieren so Messdaten `y`. In der Praxis kennen wir nur `y`, und müssen daraus `x`rekonstruieren, wir müssen also `y` ableiten. Aber was passiert, wenn die Messdaten in `y` fehlerbehaftet sind?

In [ ]:
def true_function(t):
    """
    Diese Funktion produziert die Werte der ground truth x, die wir später rekonstruieren wollen.
    """
    return (
        np.sin(3 * t) * np.exp(-(t**2) / 2)
        + 0.5 * np.sin(10 * t) * np.exp(-((t - 2) ** 2) / 0.5)
        + 0.3 * (t > 1) * (t < 1.5)
    )

In [ ]:
def integral_forward_model(x, h):
    """
    Approximiert die Integral-/Stammfunktion einer Funktion

    Args:
        x: Werte der zu integrierenden Funktion auf einem Gitter
        h: Schrittweite des Gitters (konstant vorausgesetzt)

    Returns:
        y: Kumulatives Integral der Werte in x
    """
    return np.append(0, h * np.cumsum(x))

In [ ]:
def numerical_derivative(y, h):
    """
    Numerische Ableitung (Finite Vorwärtsdifferenzen)

    Args:
        y: Funktionswerte auf äquidistantem Gitter
        h: Schrittweite des Gitters (konstant vorausgesetzt)

    Returns:
        x: numerische Ableitung
    """
    return np.diff(y) / h

In [ ]:
# Gitter der Funktionsauswertungen
n_grid = 200
t = np.linspace(-2, 4, n_grid)
h = t[1] - t[0]

# Ground truth = Auswertung der Funktion auf dem Gitter
x = true_function(t)

# Fehlerfreie Daten
y_clean = integral_forward_model(x, h)

# Verschiedene Standardabweichungen zum Testen von Messfehlern
noise_levels = [0.001, 0.01, 0.05]


fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle(
    "Ableitung einer Funktion mit verrauschten Daten", fontsize=16
)
for i, noise_level in enumerate(noise_levels):
    # Versehe Daten künstlich mit kleinen Messfehlern
    y_noisy = y_clean + noise_level * rng.normal(size=y_clean.shape)

    # Berechne Ableitung
    x_unreg = numerical_derivative(y_noisy, h)

    ###### ab hier plotten wir nur Ergebnisse:
    # Plot 1: Messdaten
    axes[0, i].plot(t, y_clean[1:], "b-", label="Exakte Daten", linewidth=2)
    axes[0, i].plot(
        t,
        y_noisy[1:],
        "r--",
        label=f"Verrauschte Daten ({noise_level})",
        alpha=0.7,
    )
    axes[0, i].set_title(f"Messdaten (noise level={noise_level})")
    axes[0, i].set_xlabel("t")
    axes[0, i].set_ylabel("y")
    axes[0, i].legend()
    axes[0, i].grid(True, alpha=0.3)

    # Plot 2: Ableitungen ohne Regularisierung
    axes[1, i].plot(t, x, "b-", label="Ground truth", linewidth=2)
    axes[1, i].plot(t, x_unreg, "r-", label="Unregularisierte Ableitung", alpha=0.7)
    axes[1, i].set_title(f"Unregularisierte Ableitung (noise level={noise_level})")
    axes[1, i].set_xlabel("t")
    axes[1, i].set_ylabel("x")
    axes[1, i].legend()
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Wie man sieht, sorgen schon minimale Änderungen der Funktionswerte (`noise_level=0.001`) für instabile Ableitungen. Bei einem `noise_level` von etwa 5% der maximalen Funktionswerte ist die Ableitung völlig unbrauchbar. Wie kann man das lösen?

In [ ]:
def regularized_derivative(y, h, sigma=1.0):
    """
    Berechnet eine regularisierte Ableitung nach Glätten der Daten

    Args:
        y: Funktionswerte auf äquidistantem Gitter t0 < t1 < ... < tN
        h: Schrittweite
        sigma: Glättungsparameter (größer = mehr Glättung)

    Returns:
        x_reg: Regularisierte Ableitung
    """
    # Hinweis: Seht euch die import statements in der ersten Zelle des notebooks an.
    x_reg = todo()

    return x_reg

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(12, 8))
fig.suptitle(
    "Regularisierte Ableitung einer Funktion mit verrauschten Daten", fontsize=16
)
for i, noise_level in enumerate(noise_levels):
    # Versehe Daten künstlich mit kleinen Messfehlern
    y_noisy = y_clean + noise_level * rng.normal(size=y_clean.shape)

    # Berechne Ableitung
    x_unreg = numerical_derivative(y_noisy, h)
    x_reg = regularized_derivative(y_noisy, h, sigma=3.0)

    # Plot 1: Messdaten
    axes[0, i].plot(t, y_clean[1:], "b-", label="Exakte Daten", linewidth=2)
    axes[0, i].plot(
        t,
        y_noisy[1:],
        "r--",
        label=f"Verrauschte Daten (sigma={noise_level})",
        alpha=0.7,
    )
    axes[0, i].set_title(f"Messdaten (Noise sigma={noise_level})")
    axes[0, i].set_xlabel("t")
    axes[0, i].set_ylabel("y")
    axes[0, i].legend()
    axes[0, i].grid(True, alpha=0.3)

    # Plot 2: Ableitungen ohne Regularisierung
    axes[1, i].plot(t, x, "b-", label="Ground truth", linewidth=2)
    axes[1, i].plot(t, x_unreg, "r-", label="Unregularisierte Ableitung", alpha=0.7)
    axes[1, i].set_title("Unregularisierte Ableitung")
    axes[1, i].set_xlabel("t")
    axes[1, i].set_ylabel("x")
    axes[1, i].legend()
    axes[1, i].grid(True, alpha=0.3)

    # Plot 3: Ableitungen mit Regularisierung
    axes[2, i].plot(t, x, "b-", label="True function", linewidth=2)
    axes[2, i].plot(t, x_reg, "g-", label="Regularisierte Ableitung", alpha=0.7)
    axes[2, i].set_title("Regularisierte Ableitung")
    axes[2, i].set_xlabel("t")
    axes[2, i].set_ylabel("x")
    axes[2, i].legend()
    axes[2, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# HALT. 
Hier bitte erstmal noch nicht weiterlesen.
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
*******
Wir wollen jetzt noch ein bisschen genauer verstehen, wie sich die Schlechtgestelltheit genau beim Lösen des Problems ausdrückt und zu numerischen Fehlern führt.
Das Ableiten von $y$ zu $x$ ist ein endlich-dimensionales Problem, also nicht ill-posed im Sinne der Definition von Hadamard. Das Problem lässt sich schreiben als simples lineares Gleichungssystem:
$$ y = Ax,\quad A = \begin{pmatrix} 
0&&\dots&0\\
h&0&\dots&0\\
h&h&\ddots&\vdots\\
\vdots&&\ddots&0\\
h&\dots&&h
\end{pmatrix} $$

In [ ]:
A = todo()

U,S,V = np.linalg.svd(A, full_matrices=False)
print(f"Größter Singulärwert von A ist {S.max():.2}, kleinster ist {S.min():.2}.")
cond_A = S.max() / S.min()
print(f"Die 2-Kondition des linearen Problems y = Ax ist also {cond_A:.2}.")

# Plot aller Singulärwerte
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.plot(S)
plt.show()

Weiterführende Fragen:
- Was passiert, wenn sich die Anzahl der Gitterpunkte, also die Dimension des Problems ändert?
- Wie passt das zum unendlich-dimensionalen Fall?
- Wie sehen die Singulärwerte im geglätteten/regularisierten Fall aus?